# Atomic models for image simulation with ASE <a id='top'></a>

This notebook introduces the Atomic Simulation Environment ([ASE](https://wiki.fysik.dtu.dk/ase/)) for creating atomic models for image simulation.

ASE is a set of tools and Python modules for setting up, manipulating and visualizing atomic structures, which is used in conjunction with a large number of atomistic simulation codes, for example [GPAW](https://wiki.fysik.dtu.dk/gpaw/) for running DFT simulations. In this notebook, ASE is introduced in the context of running electron microscopy image simulations with [*ab*TEM](https://abtem.github.io/doc/intro.html). 

### Contents

1. <a href='#the_atoms_object'> The Atoms object
2. <a href='#importing_structures'> Importing structures from files
3. <a href='#visualization'> Visualization
4. <a href='#manipulating'> Manipulating atoms
5. <a href='#exporting_structures'> Exporting structures to files
6. <a href='#orthogonal'> Orthogonal and periodic supercells

In [ ]:
%matplotlib inline

import abtem
import ase
import matplotlib.pyplot as plt
import numpy as np
from ase.visualize import view

##  The `Atoms` object <a id='the_atoms_object'></a>

The `Atoms` object defines a collection of atoms. To define `Atoms` from scratch, we need to specify at least three things:

* atomic positions,
* atomic numbers,
* a periodic cell.

Here, we create a basic model of the N<sub>2</sub> molecule.

In [ ]:
atoms = ase.Atoms("N2", positions=[(0.0, 0.0, 0.0), (1.0, 0.0, 0.0)], cell=[6, 6, 6])

__Note__: *ab*TEM and ASE uses the same [unit conventions](https://wiki.fysik.dtu.dk/ase/ase/units.html), as defined in the `ase.units` module. Thus, electron volts (eV), Ångström (Å), and atomic mass units are defined as 1.0.

We can access the corresponding properties as attributes of the object we've defined.

In [ ]:
atoms.positions

In [ ]:
atoms.numbers

In [ ]:
atoms.cell

In [ ]:
atoms.cell

The `Atoms` can be modified by directly changing the underlying NumPy arrays. For instance, we can create NO by changing the atomic number of one of the N atoms.

In [ ]:
atoms.numbers

In [ ]:
atoms.numbers[0] = 8

We can further add an additional N atom to create nitrous oxide; convenient arithmetic operations work for the `Atoms` object.

In [ ]:
atoms += ase.Atoms("N", positions=[(2.0, 0, 0)])

atoms

## Visualization <a id='visualization'></a>

We can visualize the atoms using the Matplotlib backend with *ab*TEM's `show_atoms` function, which shows a 2D projection of the structure perpendicular to a specified plane (by default $xy$, ie. perpendicular to the propagation direction).

In [ ]:
abtem.show_atoms(atoms, plane='xy');

The default ASE GUI, an interactive 3D viewer, may be started using the `view` function.

In [ ]:
from ase.visualize import view

view(atoms);

## Building a crystal

ASE has many, many tools for building various molecules, crystals, surfaces and nanostructures (see [ASE documentation](https://wiki.fysik.dtu.dk/ase/ase/build/build.html)). Here, we build the unit cell of strontium titanate.

In [ ]:
atom_pos = [(0.0, 0.0, 0.0), (0.5, 0.5, 0.5), (0.5, 0.5, 0.0)]
srtio3 = ase.spacegroup.crystal(['Sr','Ti','O'], atom_pos, spacegroup=221, cellpar=3.905, size=(1, 1, 1))

abtem.show_atoms(srtio3*(3,3,3), legend=True);

## Manipulating atoms <a id='manipulating'></a>
*ab*TEM always assumes that the imaging electrons propagate along the $z$-axis in the direction from _negative to positive_ coordinate values. Hence, to choose the zone axis, we need to manipulate the atoms so they are properly aligned.

ASE has so many tools for manipulating structures that we can't cover all of them here. As an example, we will look at the `surface` function, which can be used for creating a periodic surface (aligned with the $z$-axis) for a given set of Miller indices.

Below we orient the strontium titanate structure along the (110)-direction.

In [ ]:
from ase.build import surface

srtio3_110 = surface(srtio3, indices=(1, 1, 0), layers=2, periodic=True)

abtem.show_atoms(srtio3_110, plane="xy", legend=True);

Some of the atoms on the edge of the cell may look a little funny, but since the structure is periodic, that doesn't really matter. However, we can use the `wrap` function to wrap those back across the cell boundary.

In [ ]:
srtio3_110.wrap()

abtem.show_atoms(srtio3_110, plane="xy", legend=True, show_periodic=True);

Simulations may require a larger crystal, to repeat the atoms by 3 in $x$, 4 in $y$ and 10 in the $z$-direction, we simply multiply the atoms.

In [ ]:
repeated_srtio3 = srtio3_110.copy()

repeated_srtio3 *= (3, 4, 10)

abtem.show_atoms(repeated_srtio3, legend=True);

To view the structure from the side, we can give a different `plane` keyword. We now see that that the model is slab a few nanometers in thickness.

In [ ]:
abtem.show_atoms(repeated_srtio3, legend=True, plane='xz');

The positions and atomic numbers are just `NumPy` arrays and hence can be modified in- place. Below, we create an SrTiO<sub>3</sub>/LaTiO<sub>3</sub> interface by changing the atomic numbers of the Sr atoms with a $y$-coordinate less than $7.5 \ \mathrm{Å}$.

Note that it is important to make a copy of the structure so that you avoid modifying the original object!

In [ ]:
sto_lto = repeated_srtio3.copy()

mask = sto_lto.symbols == "Sr"

mask = mask * (sto_lto.positions[:, 1] < 7.5)

sto_lto.numbers[mask] = 57

Next, we center the atoms in the cell and add $5 \ \mathrm{Å}$ of vacuum at the entrance and exit surfaces along the $z$-axis.

In [ ]:
sto_lto.center(axis=2, vacuum=5)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
abtem.show_atoms(sto_lto, ax=ax1)
abtem.show_atoms(sto_lto, ax=ax2, plane="yz", legend=True);

## Exporting structures to files <a id='exporting_structures'></a>

The structures can also be exported in all the common atomic structure formats. Here, we export the manipulated structure as `.cif`, so we can use it in the next tutorial. The ASE IO writer will infer the file format from the ending, and supports almost anything you can imagine: https://wiki.fysik.dtu.dk/ase/ase/io/io.html

In [ ]:
from ase.io import write

write("sto_lto.cif", sto_lto)

Reading the structure is equally easy, and ASE will automatically infer the format from the file ending. (One notable exception that you may encounter in TEM are Prismatic/Computem .xyz files, which have a slightly different syntax and thus their own reader: https://wiki.fysik.dtu.dk/ase/ase/io/formatoptions.html#prismatic.)

In [ ]:
sto_lto = ase.io.read("sto_lto.cif")

## Orthogonal and periodic supercells <a id='orthogonal'></a>

The multislice algorithm requires an orthogonal periodic atomic structure as its input. However, taking any arbitrary structure and making it periodic and orthogonal is often not trivial. *ab*TEM has some tools for solving this problem.

To demonstrate this, we create a graphene structure with the minimal hexagonal unit cell. 

In [ ]:
graphene = ase.build.graphene(vacuum=4)

abtem.show_atoms(graphene * (4,4,1));

Applying `orthogonalize_cell`, we find the smallest orthogonal version of a cell. 

In [ ]:
orthogonal_graphene, transform = abtem.orthogonalize_cell(graphene, return_transform=True)

abtem.show_atoms(orthogonal_graphene * (5,3,1));

In [ ]:
from abtem.atoms import pretty_print_transform
pretty_print_transform(transform)

The problem of creating orthogonal cells is not always as trivial as for graphene. For those interested in more advanced uses of the `orthogonalize_cell` function, we have a [tutorial](https://abtem.github.io/doc/user_guide/tutorials/advanced_atomic_models.html) dedicated to the subject.

### Twisted strontium titanate

In [ ]:
abtem.show_atoms(srtio3, plane='xz')

In [ ]:
repeated_srtio3 = srtio3 * (1, 1, 8)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 4))
abtem.show_atoms(repeated_srtio3, ax=ax1, title="Beam view")
abtem.show_atoms(repeated_srtio3, ax=ax2, plane="xz", title="Side view")
fig.tight_layout();

In [ ]:
rotated_srtio3_1 = repeated_srtio3.copy()
rotated_srtio3_1.rotate(8.8/2,"z", rotate_cell=True)

rotated_srtio3_2 = repeated_srtio3.copy()
rotated_srtio3_2.rotate(-8.8/2,"z", rotate_cell=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
abtem.show_atoms(rotated_srtio3_1, title="Positive rotation", ax=ax1)
abtem.show_atoms(rotated_srtio3_2, title="Negative rotation", ax=ax2)

In [ ]:
from abtem.atoms import pretty_print_transform

maxreps = 12

atoms_top, transform1 = abtem.orthogonalize_cell(
    rotated_srtio3_1, max_repetitions=maxreps, return_transform=True
)

atoms, transform2 = abtem.orthogonalize_cell(
    rotated_srtio3_2, max_repetitions=maxreps, return_transform=True
)

atoms_bottom = atoms.copy()
atoms_bottom.translate([0,0,-atoms_top.cell[2,2]])

pretty_print_transform(transform1)
pretty_print_transform(transform2)

abtem.show_atoms(atoms, title="Beam view", plane='xz')

In [ ]:
combined = atoms_top + atoms_bottom
combined.cell[2,2] *= 2
combined.center(axis=2, vacuum=4)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
abtem.show_atoms(combined, ax=ax1, title="Beam view", scale=0.4)
abtem.show_atoms(combined, ax=ax2, plane="xz", title="Side view", scale=0.4)
fig.tight_layout();